----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content layer - IPA - FxCross stream - With events
This notebook demonstrates how to subscribe to FX Cross pricing analytics events.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [15]:
import refinitiv.data as rd
from refinitiv.data.content.ipa.financial_contracts import cross
import datetime

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [16]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7fb950153f40 {name='workspace'}>

## About events
When you use a QuantitativeDataStream object to open streaming instruments with events, you need to define callback functions that are called by a background execution thread when new events are received for the instruments you requested. There are 3 different types of functions you can use depending on the type of events you are interrested in. These callback functions are optionnal meaning that you do not have to use all of them. 

### The 3 event types and their related callback functions:
 - **Response events:** 
 
 Callback functions used to receive Response events take 3 parameters: 
  - The QuantitativeDataStream that received the event.
  - The data of the concerned instrument.
  - The field names of the data.
 
 
 - **Update events:** 
 
 Callback functions used to receive Update events take 3 parameters 
  - The QuantitativeDataStream that received the event.
  - The data of the concerned instrument.
  - The field names of the data.
 
 
 - **State events:** 
 
 Callback functions used to receive Update events take 3 parameters 
  - The QuantitativeDataStream that received the event.
  - The state of the stream.

## Retrieve data

### Define the FxCross

In [17]:
fxcross_def = cross.Definition(
    instrument_tag="USDAUD",
    fx_cross_type=cross.FxCrossType.FX_SPOT,
    fx_cross_code="USDAUD",
    fields=["InstrumentTag", "FxSpot_BidMidAsk", "ErrorCode", "Ccy1SpotDate", "Ccy2SpotDate"],
    extended_params = {
            "marketData": {
                "fxSpots": [
                    {
                        "spotDefinition": {
                            "fxCrossCode": "AUDUSD", 
                            "Source": "Composite"
                        }
                    }
                ]
            }
        }
)

### Get the stream and register callbacks to receive events

In [18]:
def display_event(eventType, data, headers):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    print(data)
    print(headers)
    
stream = (
        fxcross_def.get_stream()
        .on_response(lambda data, headers, stream: display_event("response", data, headers))
        .on_update(lambda data, headers, stream: display_event("update", data, headers))
        .on_state(lambda state, stream: display_event("state", state, None))
    )

### Open the stream

In [19]:
stream.open()

----------------------------------------------------------

<OpenState.Opened: 'Opened'>


>>> response event received at 14:26:49.302136
[['USDAUD', {'bid': 1.4494854326714017, 'ask': 1.449905756125852, 'mid': 1.449695594398627, 'processingInformation': ''}, '', '2022-06-24', '2022-06-24']]
['InstrumentTag', 'FxSpot_BidMidAsk', 'ErrorCode', 'Ccy1SpotDate', 'Ccy2SpotDate']
----------------------------------------------------------
>>> update event received at 14:26:50.046717
[['USDAUD', {'bid': 1.4494854326714017, 'ask': 1.449905756125852, 'mid': 1.449695594398627, 'processingInformation': ''}, '', '2022-06-24', '2022-06-24']]
['InstrumentTag', 'FxSpot_BidMidAsk', 'ErrorCode', 'Ccy1SpotDate', 'Ccy2SpotDate']
----------------------------------------------------------
>>> update event received at 14:26:51.029827
[['USDAUD', {'bid': 1.4496955639315745, 'ask': 1.4501160092807426, 'mid': 1.4499057866061587, 'processingInformation': ''}, '', '2022-06-24', '2022-06-24']]
['InstrumentTag', 'FxSpot_BidMidAsk', 'ErrorCode', 'Ccy1SpotDate', 'Ccy2SpotDate']
----------------------------

### Close the stream

In [20]:
stream.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [21]:
rd.close_session()